# Exploring Fraud Detection With Neo4j & Graph Data Science

This analysis uses [Neo4j and Graph Data Science (GDS)](https://neo4j.com/docs/graph-data-science/current/) to explore an anonymized data sample from a Peer-to-Peer (P2P) payment platform.  The notebook is split up into the following sections to cover various stages of the graph data science workflow:

- Notebook Setup
- Part 1: Exploring Connected Fraud Data
- Part 2: Resolving Fraud Communities using Entity Resolution and Community Detection
- Part 3: Recommending Suspicious Accounts With Centrality & Node Similarity
- Part 4: Predicting Fraud Risk Accounts with Machine Learning

## Notebook Setup <a name="p0"></a>

In [ ]:
!pip install -U graphdatascience

In [ ]:
from neo4j import GraphDatabase
import pandas as pd
import configparser
import os
pd.set_option('display.width', 0)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 50)

### Neo4j Settings
The `NEO4J_PROPERTIES_FILE` is an ini configuration file for Neo4j properties so this notebook can connect to
your Neo4j instance and load data. The ini file should be formatted as follows
```
[NEO4J]
PASSWORD=<password>
USERNAME=<database name, is 'neo4j' by default>
HOST=<host uri>
```

Set `NEO4J_PROPERTIES_FILE` to None or any non-existent location to use the below defaults
```
HOST = 'neo4j://localhost'
USERNAME = 'neo4j'
PASSWORD = 'password'
```

In [ ]:
NEO4J_PROPERTIES_FILE="gs://neo4j_voutila/neo4j.ini"

In [ ]:
from google.cloud import storage
from google.cloud.storage.blob import Blob

gcs = storage.Client()
ini = Blob.from_string(NEO4J_PROPERTIES_FILE, client=gcs)
assert ini.exists()

HOST, USERNAME, PASSWORD = None, "neo4j", "password"
try:
    config = configparser.RawConfigParser()
    text = ini.download_as_text()
    config.read_string(text)
    HOST = config['NEO4J']['HOST']
    USERNAME = config['NEO4J']['USERNAME']
    PASSWORD = config['NEO4J']['PASSWORD']
    print(f'Using custom database properties: {USERNAME}@{HOST}')
except Exception as e:
    print(e)


In [ ]:
driver = GraphDatabase.driver(HOST, auth=(USERNAME, PASSWORD))
driver.verify_connectivity()

### Helper Functions

In [ ]:
def run(driver, query, params={}):
    with driver.session() as session:
        if params:
            return [r for r in session.run(query, params)]
        return [r for r in session.run(query)]
    
def clear_graph(driver, graph_name):
    if run(driver, f"CALL gds.graph.exists('{graph_name}') YIELD exists RETURN exists")[0].get("exists"):
        run(driver, f"CALL gds.graph.drop('{graph_name}')")

def clear_all_graphs(driver):
    graphs = run(driver, 'CALL gds.graph.list() YIELD graphName RETURN collect(graphName) as graphs')[0].get('graphs')
    for g in graphs:
        run(driver, f"CALL gds.graph.drop('{g}')")

def identifier_degrees(user_label, degree_property):
    params = {'userLabel': user_label, 'property': degree_property}
    clear_graph(driver, 'id-projection')
    print(f'projecting identity graph for {user_label}')
    run(driver, '''
        CALL gds.graph.project('id-projection',
            [$userLabel, 'Card', 'Device', 'IP'],
            {
                HAS_CC: {orientation: 'REVERSE'},
                HAS_IP: {orientation: 'REVERSE'},
                USED: {orientation: 'REVERSE'}
            }
        )
    ''', params=params)

    print(f'calculating degree centrality')
    run(driver, '''
        CALL gds.degree.mutate('id-projection', {
            mutateProperty: $property
        })
    ''', params=params)

    print(f'writing degree scores to {degree_property}')
    run(driver, '''
        CALL gds.graph.writeNodeProperties('id-projection',
            [$property],
            ['Card', 'Device', 'IP']
        )
    ''', params=params)
    
    print(f'updating indexes...')
    run(driver, f'''
        CREATE INDEX card_{degree_property} IF NOT EXISTS FOR (c:Card) on c.{degree_property};
    ''')
    run(driver, f'''
        CREATE INDEX device_{degree_property} IF NOT EXISTS FOR (c:Device) on c.{degree_property};
    ''')
    run(driver, f'''
        CREATE INDEX ip_{degree_property} IF NOT EXISTS FOR (c:IP) on c.{degree_property};
    ''')
    clear_graph(driver, 'id-projection')
    print('finished.')

## Part 1: Exploring Connected Fraud Data

### Dataset Introduction

We will be using an anonymized sample of user accounts and transactions from a real-world Peer-to-Peer (P2P) platform. Prior to ingesting the data into graph, the original identification numbers were removed and categorical values were masked. Each user account has a unique 128-bit identifier, while the other nodes, representing unique credit cards, devices, and ip addresses have been assigned random UUIDs. These identifier are stored as the guid property in the graph schema.

Below is a visualization of the graph schema:

<img src="img/schema.png" width="500" >

Each user node has an indicator variable for money transfer fraud (named MoneyTransferFraud) that is 1 for known fraud and 0 otherwise. This indicator is determined by a combination of credit card chargeback events and manual review. A chargeback is an action taken by a bank to reverse electronic payments. It involves reversing a payment and triggering a dispute resolution process, often for billing errors and unauthorized credit use. In short, a user must have at least one chargeback to be considered fraudulent. Only a small proportion of the user accounts, roughly 0.7 %, are flagged for fraud.

Each user node has an indicator variable for money transfer fraud (named `MoneyTransferFraud`) that is 1 for known fraud and 0 otherwise. This indicator is determined by a combination of credit card chargeback events and manual review. A chargeback is an action taken by a bank to reverse electronic payments. It involves reversing a payment and triggering a dispute resolution process, often for billing errors and unauthorized credit use. In short, a user must have at least one chargeback to be considered fraudulent. Only a small proportion of the user accounts, roughly 0.7 %, are flagged for fraud.


Below is a breakdown of high-level counts by labels and relationships as well as the flagged accounts.

In [ ]:
# total node counts
result = run(driver, 'CALL apoc.meta.stats() YIELD labels AS nodeCounts')
pd.DataFrame(dict(result[0])).transpose()

In [ ]:
# total relationship counts
result = run(driver, 'CALL apoc.meta.stats() YIELD relTypesCount as relationshipCounts')
pd.DataFrame(dict(result[0])).transpose()

In [ ]:
result = run(driver, 'MATCH(u:User) RETURN u.fraudMoneyTransfer AS fraudMoneyTransfer, count(u) AS cnt')
pd.DataFrame(dict(record) for record in result)

### A Closer Look at Cards and Devices

Upon an initial investigation of aggregate statistics, the labeled fraud accounts do not look well connected which would naturally make one skeptical of a graph based approach.  This is exemplified well by the lack of discrimination in key identifiers.

As a first step, I break out the ratio of fraud vs non-fraud users connected to credit cards and devices below. I find that very few cards or devices are centered well around flagged accounts. This is a bit surprising for me. If a card or device is used by a flagged account, I would expect the other accounts that use the card/device to also be mostly flagged for fraud as well.

In [ ]:
# Setting a label for flagged users will enable faster lookups in cypher and faster gds projections
run(driver, 'MATCH(u:User) WHERE u.fraudMoneyTransfer=1 SET u:FlaggedUser RETURN count(u)')

In [ ]:
# Use GDS degree centrality to count the number of Users connected to each identifier type - Card, Device, IP
identifier_degrees('User', 'degree')
# Use GDS degree centrality to count the number of FLAGGED Users connected to each identifier type - Card, Device, IP
identifier_degrees('FlaggedUser', 'flaggedDegree')

# Calculate the ratio of flagged users to total users
run(driver, '''
        MATCH(n) WHERE n:Card OR n:Device OR n:IP
        SET n.flaggedRatio = toFloat(n.flaggedDegree)/toFloat(n.degree)
        RETURN count(n) AS numChanged
''')

In [ ]:
result = run(driver, '''
    MATCH(n:Card) WHERE n.degree > 1
    WITH toFloat(count(n)) AS total
    MATCH(n:Card) WHERE n.degree > 1
    WITH n, total, CASE
        WHEN n.flaggedRatio=0 THEN '0'
        WHEN n.flaggedRatio=1  THEN '1'
        ELSE 'Between 0-1' END AS flaggedUserRatio
    RETURN flaggedUserRatio, count(n) AS count, round(toFloat(count(n))/total,3) AS percentCount ORDER BY flaggedUserRatio
''')
print('Flagged User Ratio for Card Count')
pd.DataFrame([dict(record) for record in result])

In [ ]:
result = run(driver, '''
    MATCH(n:Device) WHERE n.degree > 1
    WITH toFloat(count(n)) AS total
    MATCH(n:Device) WHERE n.degree > 1
    WITH n, total, CASE
        WHEN n.flaggedRatio=0 THEN '0'
        WHEN n.flaggedRatio=1  THEN '1'
        ELSE 'Between 0-1' END AS flaggedUserRatio
    RETURN flaggedUserRatio, count(n) AS count, round(toFloat(count(n))/total,3) AS percentCount ORDER BY flaggedUserRatio
''')
print('Flagged User Ratio for Device Count')
pd.DataFrame([dict(record) for record in result])

#### Exploring Potential Fraud Patterns with Community Detection

In this example, it is unclear to me what exactly is taking place, and I have a hunch that the fraud activity is not completely labeled given the lack of connectivity and the limited chargeback logic used to flag fraud. At the same time I do not want to simply label every user that shares a card or device with another flagged account, since it is possible that a benign user's device and or card was used fraudulently by another. Since fraudsters are actively avoiding being identified, actors committing fraud are often not just represented by a single user account, but rather by multiple accounts and identifiers which, hopefully for us, share some connections and similarities.

In a graph, we can attempt to roughly identify these fragmented identities with Community Detection, a large set of methods that attempt to partition graphs into well connected groups a.k.a. Communities, where the connectivity in the communities is significantly higher than outside the community. There are multiple forms of community detection. We will use a couple in this post, starting with Louvain.

#### Using Louvain Community Detection

[Louvain](https://neo4j.com/docs/graph-data-science/current/algorithms/louvain/) is useful for exploratory analysis of communities because it uses a form of modularity scoring to split up the graph into hierarchical clusters. This means that your theories around fraud patterns and graph structure don't need to be exact for it to provide informative communities and insights.

I will leverage P2P transactions, cards, and devices for Louvain. We will leave out IP addresses for now since they have some super node issues (we will leverage IP addresses in subsequent parts ). Below are the queries for running Louvain in Neo4j GDS and aggregating community statistics. The last query below orders communities by the count of flagged users so we can further examine some of the more concentrated flagged communities.


In [ ]:
clear_graph(driver, 'comm-projection')

run(driver, '''
    CALL gds.graph.project('comm-projection', ['User','Card', 'Device'], {
        HAS_CC: {orientation: 'UNDIRECTED'},
        USED: {orientation: 'UNDIRECTED'},
        P2P: {orientation: 'NATURAL', aggregation: 'SINGLE'}
        }
    )
''')

run(driver, '''
    CALL gds.louvain.write('comm-projection', {
            writeProperty: 'louvainCommunityId'
    })
''')

In [ ]:
result  = run(driver, '''
    MATCH (u:User)
    WITH u.louvainCommunityId AS community,
        count(u) AS cnt,
        sum(u.fraudMoneyTransfer) as flaggedCount
    RETURN community,
        cnt,
        flaggedCount,
        toFloat(flaggedCount)/toFloat(cnt) AS flaggedRatio
    ORDER BY flaggedCount DESC LIMIT 100
''')
print("Louvain Communities Ordered by count of Flagged Users")
pd.DataFrame([dict(record) for record in result])

We can view communities of users and connecting identifiers in [Neo4j Bloom](https://neo4j.com/docs/bloom-user-guide/current/) or Neo4j Browser with queries of the form:

```cypher
MATCH(u1:User{louvainCommunityId: $id})-[r1:HAS_CC|HAS_IP|USED]->(n)<-[r2:HAS_CC|HAS_IP|USED]-(u2:User{louvainCommunityId: $id})
WITH *
OPTIONAL MATCH (u1)-[:P2P]-(u2)
RETURN *
```
Below are a couple examples of these communities. Flagged users are colored red with caption=0 with other users colored orange and caption=1. We will see an interesting pattern of flagged users sending money to non-flagged users with which they share devices, cards, and IPs. This behavior is suspicious because it suggests that the same individual or group of individuals may be using multiple accounts to withdraw cash from credit transactions prior to a chargeback being executed on the cards. The shared credit card examples (such as example 2 below) are some of the most indicative.

Example 1:
<img src="img/louvain-community-11.png" width="900" >

Example 2:
<img src="img/louvain-community-12.png" width="900" >

This pattern indicates that the receiving user accounts are fraud risk accounts. In the next section we will use these patterns to inform resolution of fraud communities and the labeling of additional fraud risk accounts.

## Part 2: Resolving Fraud Communities using Entity Resolution and Community Detection

Identifying communities that reflect underlying groups of individuals is often a key step to fraud detection.  We already explored with Louvain; in this section, in this part, we will provide more formal definitions for resolving entities that will allow us to partition well defined communities in a scalable manner.

To accomplish this, we will define some Entity Resolution (ER) rules that will allow us to draw relationships between users which we believe belong to the same underlying community. We will then use the [Weakly Connected Components (WCC)](https://neo4j.com/docs/graph-data-science/current/algorithms/wcc/) algorithm to resolve the communities. Lastly, we will label all users in communities that include flagged accounts as fraud risks.

### Entity Resolution Business Rules

We will now use Entity Resolution (ER) to resolve groups of individuals behind sets of user accounts. For this analysis, we will use some pretty straightforward ER business logic. If either of the two below conditions are true, we will resolve two user accounts by linking them together with a new relationship type.

1. One user sent money to another user that shares the same credit card
2. Two users share a card or device connected to less than or equal to 10 total accounts, and those two users also share at least two other identifiers of type credit card, device, or IP address

You could switch out or add different rules to the above, these are just examples. In a real-world scenario these business rules would pass by SMEs and possibly be backed by further supervised machine learning on manually labeled data. More advanced techniques for this type of ER are possible in graph and we describe them in [this whitepaper](https://neo4j.com/whitepapers/graph-data-science-use-cases-entity-resolution/) and [this blog](https://neo4j.com/developer-blog/exploring-supervised-entity-resolution-in-neo4j/).

For a P2P dataset, we do not necessarily want to label all senders/receivers of flagged user transactions as fraudulent since some fraud schemes involve transactions with victims. Furthermore, additional identifiers such as IP may be inexact and cards + devices can be fraudulently controlled/used without the owners permission. Hence I used somewhat stringent rules that aligned with the patterns noted in part 1. We can apply relationships to reflect these business rules using cypher:


In [ ]:
# P2P with shared card rule
run(driver, '''
    MATCH (u1:User)-[r:P2P]->(u2)
    WITH u1, u2, count(r) AS cnt
    MATCH (u1)-[:HAS_CC]->(n)<-[:HAS_CC]-(u2)
    WITH u1, u2, count(DISTINCT n) AS cnt
    MERGE(u1)-[s:P2P_WITH_SHARED_CARD]->(u2)
    RETURN count(DISTINCT s) AS cnt
''')

In [ ]:
# Shared ids rule
run(driver, '''
    MATCH (u1:User)-[:HAS_CC|USED]->(n)<-[:HAS_CC|USED]-(u2)
    WHERE n.degree <= 10 AND id(u1) < id(u2)
    WITH u1, u2, count(DISTINCT n) as cnt
    MATCH (u1)-[:HAS_CC|USED|HAS_IP]->(m)<-[:HAS_CC|USED|HAS_IP]-(u2)
    WITH u1, u2, count(DISTINCT m) as cnt
    WHERE cnt > 2
    MERGE(u1)-[s:SHARED_IDS]->(u2)
    RETURN count(DISTINCT s)
''')

### Using Weakly Connected Components (WCC) to Resolve Communities

[Weakly Connected Components (WCC)](https://neo4j.com/docs/graph-data-science/current/algorithms/wcc/) is a practical and highly scalable community detection algorithm. It is also deterministic and very explainable. It defines a community simply as a set of nodes connected by a subset of relationship types in the graph. This makes WCC a good choice for formal community assignment in production fraud detection settings.

Below we run WCC on users via the ER relationships created above:


In [ ]:
clear_graph(driver, 'comm-projection')
run(driver, '''
    CALL gds.graph.project('comm-projection',
        ['User'],
        {
            SHARED_IDS: {orientation: 'UNDIRECTED'},
            P2P_WITH_SHARED_CARD: {orientation: 'UNDIRECTED'}
        }
    )
''')

run(driver, '''
    CALL gds.wcc.write('comm-projection', {
        writeProperty: 'wccId'
    }) YIELD writeMillis,
        nodePropertiesWritten,
        componentCount,
        componentDistribution
''')

### Labeling Fraud Risk User Accounts

As these communities are meant to label underlying groups of individuals, if even one flagged account is in the community, we will label all user accounts in the group as fraud risks:


In [ ]:
run(driver, '''
    MATCH (f:FlaggedUser)
    WITH collect(DISTINCT f.wccId) AS flaggedCommunities
    MATCH(u:User) WHERE u.wccId IN flaggedCommunities
    SET u:FraudRiskUser
    SET u.fraudRisk=1
    RETURN count(u)
''')

In [ ]:
run(driver, '''
    MATCH (u:User) WHERE NOT u:FraudRiskUser
    SET u.fraudRisk=0
    RETURN count(u)
''')

### WCC Community Statistics

The breakdown of communities by size is listed below. The majority are single user communities. Only a small portion have multiple users and of those, community sizes are mostly 2 and 3. Larger communities are rare. However, if we look at the fraudUser accounts we will see that the majority reside in multi-user communities. The 118 fraud accounts in single user communities are flagged users (via original chargeback logic) that have yet to be resolved to a community.

In [ ]:
result = run(driver, '''
    MATCH (u:User)
    WITH u.wccId AS community, count(u) AS cSize, sum(u.fraudRisk) AS cFraudSize
    WITH community, cSize, cFraudSize,
    CASE
        WHEN cSize=1 THEN ' 1'
        WHEN cSize=2 THEN ' 2'
        WHEN cSize=3 THEN ' 3'
        WHEN cSize>3 AND cSize<=10 THEN ' 4-10'
        WHEN cSize>10 AND cSize<=50 THEN '11-50'
        WHEN cSize>10 THEN '>50' END AS componentSize
    RETURN componentSize, count(*) as numberOfComponents, sum(cSize) AS totalUserCount, sum(cFraudSize) AS fraudUserCount ORDER BY componentSize
''')
pd.DataFrame([dict(record) for record in result])

Similar to Louvain we can view communities of users and connecting identifiers in [Neo4j Bloom](https://neo4j.com/docs/bloom-user-guide/current/) or Browser with queries of the form:

```cypher
MATCH(u1:User{wccId: $id})-[r1:HAS_CC|HAS_IP|USED]->(n)<-[r2:HAS_CC|HAS_IP|USED]-(u2:User{wccId: $id})
WITH *
OPTIONAL MATCH (u1)-[r3:P2P]-(u2)
RETURN *
```
Below are a couple examples. Users that were flagged via initial chargeback logic are colored red with caption=1, while other users are colored orange with caption=0. Overall, you will notice a high degree of overlapping connectivity of identifiers and P2P transactions between users, which we should expect given our ER rules.

Example 1:
<img src="img/wcc-community-12.png" width="900" >

Example 2:
<img src="img/wcc-community-11.png" width="900" >

### Outcomes of Fraud Risk Labeling
Fraud Risk labeling helped identify an additional 211 new fraud risk user accounts, nearly doubling the number of known fraud users (87.5% increase). We also see that 65% of the money going to/from previously flagged accounts and other users can be attributed to the newly identified risk accounts:

In [ ]:
run(driver, '''
   MATCH (:FlaggedUser)-[r:P2P]-(u)  WHERE NOT u:FlaggedUser
   WITH toFloat(sum(r.totalAmount)) AS p2pTotal
   MATCH (u:FraudRiskUser)-[r:P2P]-(:FlaggedUser) WHERE NOT u:FlaggedUser
   WITH p2pTotal,  toFloat(sum(r.totalAmount)) AS fraudRiskP2pTotal
   RETURN round((fraudRiskP2pTotal)/p2pTotal,3) AS p
''')[0].get('p')

Additionally, while the newly identified 211 accounts represents less than 1% of total users in the sample, 12.7% of the total P2P amount in the sample involved the newly identified accounts as senders or receivers:

In [ ]:
run(driver, '''
   MATCH (:User)-[r:P2P]->()
   WITH toFloat(sum(r.totalAmount)) AS p2pTotal
   MATCH (u:FraudRiskUser)-[r:P2P]-() WHERE NOT u:FlaggedUser
   WITH p2pTotal, toFloat(sum(r.totalAmount)) AS fraudRiskP2pTotal
   RETURN round((fraudRiskP2pTotal)/p2pTotal,3) AS p
''')

Finally, we can see an improvement in card and device discrimination with many more cards and devices being used by fraud risk accounts exclusively.

In [ ]:
identifier_degrees('FraudRiskUser', 'fraudRiskDegree')

run(driver, '''
        MATCH(n) WHERE n:Card OR n:Device OR n:IP
        SET n.fraudRiskRatio = toFloat(n.fraudRiskDegree)/toFloat(n.degree)
''')

In [ ]:
result = run(driver, '''
    MATCH(n:Card) WHERE n.degree > 1
    WITH toFloat(count(n)) AS total
    MATCH(n:Card) WHERE n.degree > 1
    WITH n, total, CASE
        WHEN n.fraudRiskRatio=0 THEN '0'
        WHEN n.fraudRiskRatio=1  THEN '1'
        ELSE 'Between 0-1' END AS fraudRiskRatio
    RETURN fraudRiskRatio, count(n) AS count, round(toFloat(count(n))/total,3) AS percentCount ORDER BY fraudRiskRatio
''')
pd.DataFrame([dict(record) for record in result])

In [ ]:
result = run(driver, '''
    MATCH(n:Device) WHERE n.degree > 1
    WITH toFloat(count(n)) AS total
    MATCH(n:Device) WHERE n.degree > 1
    WITH n, total, CASE
        WHEN n.fraudRiskRatio=0 THEN '0'
        WHEN n.fraudRiskRatio=1  THEN '1'
        ELSE 'Between 0-1' END AS fraudRiskRatio
    RETURN fraudRiskRatio, count(n) AS count, round(toFloat(count(n))/total,3) AS percentCount ORDER BY fraudRiskRatio
''')
pd.DataFrame([dict(record) for record in result])

The aggregate P2P statistics combined with improvements in Card and Device metrics are significant given the limited scope of the previously flagged fraud which focused on chargebacks.  These results strongly imply that there are more sophisticated networks of fraudulent money flows behind the chargebacks rather than the chargebacks being isolated occurrences.

## Part 3: Recommending Suspicious Accounts With Centrality & Node Similarity

In parts 1 & 2 we explored the graph and identified high risk fraud communities. At this stage, we may want to expand beyond our business logic to automatically identify other users that are suspiciously similar to the fraud risks already identified. Neo4j and GDS makes it simple to triage and recommend such suspect users in a matter of seconds. We can leverage both centrality and similarity algorithms for this.


### Using Weighted Degree Centrality to Recommend Potential High Risk Accounts

We can quickly and easily generate a ranked list of suspicious user accounts with weighted degree centrality. Specifically, we can calculate the degree centrality of users in respect to their identifiers (Devices, Cards, and IPs) weighted by the fraudRiskRatios we made in part 2. In this case, a simple Cypher query suffices.


In [ ]:
result = run(driver,'''
    MATCH(f:FraudRiskUser)-[:HAS_CC|HAS_IP|USED]->(n)
    WITH DISTINCT n
    MATCH(u:User)-[:HAS_CC|HAS_IP|USED]->(n) WHERE NOT u:FraudRiskUser
    WITH left(u.guid,8) as uid,
        sum(n.fraudRiskRatio) AS totalIdFraudRisk,
        count(n) AS numberFraudRiskIds
    WITH uid, totalIdFraudRisk,
        numberFraudRiskIds,
        totalIdFraudRisk/toFloat(numberFraudRiskIds) AS averageFraudIdRisk
    WHERE averageFraudIdRisk >= 0.25
    RETURN uid, totalIdFraudRisk, numberFraudRiskIds, averageFraudIdRisk
    ORDER BY totalIdFraudRisk DESC LIMIT 10
''')

pd.DataFrame([dict(record) for record in result])

Users in the above result list are sorted by how much identifying information they share with previously labeled fraud risks, the ones with the most being at the top. Technically speaking, these users are ranked by their total Id fraud risk, which is equal to the sum of the fraudRiskRatios from the Identifiers they are connected to. In the query I also implement a limit on the average fraud risk to avoid users that just have a lot of high-degree identifiers (likely proxy ip addresses shared by only a small fraction of fraud risk users). This sort of filtering can be tweaked by use case to get the right balance between total risk vs average risk.

In a real-world fraud detection use case, these results can be triaged by analysts to label more fraud accounts and grow labeled fraud communities.

### Using Node Similarity to Expand on Fraud Communities

Simple calculations like weighted degree centrality work well for identifying suspicious users over the whole graph, but what if we are interested in how users are related to a specific fraud risk community or set of communities? Perhaps we hypothesize that communities of fraud risk users are actually bigger than currently represented but we don't have exact business rules to apply. We can leverage similarity algorithms to help us score and recommend users for this.

GDS offers multiple algorithms for similarity. In this analysis I will focus on the aptly named  Node Similarity algorithm – you can read more about it [here](https://neo4j.com/docs/graph-data-science/current/algorithms/node-similarity/). Node similarity parallelizes well and is explainable. It identifies pairs of similar nodes based on a straightforward Jaccard similarity calculation. So while other ML-based similarity approaches like FastRP + KNN covered [in this post](https://neo4j.com/developer-blog/exploring-practical-recommendation-systems-in-neo4j/) scale well for running globally on very large graphs, Node Similarity is a good choice where explainability is important and you can narrow down the universe of comparisons to a subset of your data. Examples of narrowing down include focusing on just single communities, newly added users, or users within a specific proximity to suspect accounts. In this analysis we will take the third approach, filtering to just those Cards, Devices, and IP addresses that connect to at least one fraud risk account from part 2.


Below I apply three queries to calculate node similarity. The first query enables the identifier filtering via setting a new label on the Card, Device, and IP address nodes that connect to fraud risk accounts. The first query also weights relationships by the inverse of degree centrality, essentially downplaying the importance of identifiers proportional to the number of other users they connect to. This is important as some identifiers, particularly IP addresses, can connect to hundreds or thousands of users, in which case the identifier may be very generic (like a proxy IP address) and not as relevant to true user identity. The second and third query project the graph and write relationships back to the database with a score to represent similarity strength between user node pairs. You will notice that I use a similarity cutoff of 0.01 in the third query, which is intended to rule out weak associations and keep the similarities relevant.


In [ ]:
# label identifiers and users that are close to fraud risk users and assign inverse degree weight
run(driver,'''
    MATCH(f:FraudRiskUser)-[:HAS_CC|HAS_IP|USED]->(n)
    WITH DISTINCT n
    MATCH(n)<-[r:HAS_CC|HAS_IP|USED]-(u)
    SET n:FraudSharedId
    SET r.inverseDegreeWeight = 1.0/(n.degree-1.0)
    RETURN count(DISTINCT n)
''')

In [ ]:
clear_graph(driver, 'similarity-projection')
run(driver, '''
    CALL gds.graph.project('similarity-projection',
        ['User', 'FraudSharedId'],
        ['HAS_CC', 'USED', 'HAS_IP'],
        {relationshipProperties: ['inverseDegreeWeight']}
    ) YIELD nodeCount, relationshipCount
''')

In [ ]:
run(driver, '''
    CALL gds.nodeSimilarity.write('similarity-projection',{
        writeRelationshipType: 'SIMILAR_IDS',
        writeProperty: 'score',
        similarityCutoff: 0.01,
        relationshipWeightProperty: 'inverseDegreeWeight'
    }) YIELD relationshipsWritten, similarityDistribution
''')

From there, we can run a Cypher query to rank users by how similar they are to known fraud risk communities.

In [ ]:
#get nodes similar to the high risk ones
result = run(driver, '''
    MATCH (f:FraudRiskUser)
    WITH f.wccId AS componentId, count(*) AS numberOfUsers, collect(f) AS users
    UNWIND users AS f
    MATCH (f)-[s:SIMILAR_IDS]->(u:User) WHERE NOT u:FraudRiskUser AND numberOfUsers > 2
    RETURN u.guid AS userId, sum(s.score) AS totalScore, collect(DISTINCT componentId) AS closeToCommunityIds ORDER BY totalScore DESC
''')
pd.DataFrame([dict(record) for record in result])

Let’s take a look at the first user in the list, user `0b3f278f`, in [Bloom](https://neo4j.com/docs/bloom-user-guide/current/) . This user account seems interesting in the sense that they connect to two different communities.

<img src="img/node-similarity-1.png" width="700" >

You can see how this user connects to the two fraud risk communities and how the similarity relationships were based on shared IP addresses. This user seems to act as a sort of bridge between the two communities, suggesting not only that the user is likely part of the fraud communities but also that the two communities may actually reflect one-in-the same.


Overall, centrality and similarity metrics like these are fast and easy to implement with Neo4j and GDS. They can help advance your Data Science approach by introducing automated and semi-supervised processes to assist in targeted triage and identification of suspicious user accounts based on previously labeled data.

## Part 4: Predicting Fraud Risk Accounts with Machine Learning

In real-world scenarios we often don't know which user accounts are fraudulent ahead of time. There will be cases, like with this dataset, where some accounts get flagged due to business rules, for example chargeback history, or via user reporting mechanisms. However, as we saw in parts 1 & 2 above, those flags don't tell the whole story. The community detection and recommendation approaches we previously covered can go a very long way in helping us understand this story and label additional fraud risk users and patterns. That said, there are multiple reasons why we may want to add supervised Machine Learning to predict the additional fraud risks:

 - __Proactive Detection:__ We can train a model to identify fraudulent actors ahead of time (such as before additional chargebacks or system flags) and better identify new communities that aren't connected to older known fraud accounts.
 - __Measurable Performance:__ Supervised learning models produce clear performance metrics that enable us to evaluate and adjust as needed
 - __Automation:__ supervised Machine Learning automates the prediction of fraud risk accounts.

In the below sections we will walk through how to engineer graph features for ML, export those features to python, then train and evaluate an ML model for fraud classification.

### Feature Engineering
If we want a machine learning model to successfully classify fraud risk user accounts, we need to supply features that will be informative for the task. The below commands engineer graph features using GDS.  This includes features from [WCC](https://neo4j.com/docs/graph-data-science/current/algorithms/wcc/) community sizes, [pageRank](https://neo4j.com/docs/graph-data-science/current/algorithms/page-rank/), and [degree centrality](https://neo4j.com/docs/graph-data-science/current/algorithms/degree-centrality/).

#### Community Features

In [ ]:
run(driver, '''
    MATCH (u:User)
    WITH u.wccId AS componentId, count(*) AS communitySize, collect(u) AS users
    WITH communitySize, toInteger(communitySize > 1) AS partOfCommunity, users
    UNWIND users as u
    SET u.communitySize = communitySize
    SET u.partOfCommunity = partOfCommunity
    RETURN count(u)
''')

#### P2P With Cards and Id Sharing Centrality Features

In [ ]:
clear_graph(driver,'user-feature-projection')
run(driver, '''
    CALL gds.graph.project('user-feature-projection', ['User'], {
        P2P: {type: 'P2P', orientation: 'NATURAL', aggregation: 'SUM', properties: ['totalAmount']},
        P2P_REVERSE: {type: 'P2P', orientation: 'REVERSE', aggregation: 'SUM', properties: ['totalAmount']},
        SHARED_IDS: {type: 'SHARED_IDS', orientation: 'UNDIRECTED'},
        P2P_WITH_SHARED_CARD: {type: 'P2P_WITH_SHARED_CARD', orientation: 'NATURAL'},
        P2P_WITH_SHARED_CARD_REVERSE: {type: 'P2P_WITH_SHARED_CARD', orientation: 'REVERSE'}
    })
''')

In [ ]:
run(driver, '''
    CALL gds.degree.write('user-feature-projection', {
        relationshipTypes: ['SHARED_IDS'],
        writeProperty: 'sharedIdsDegree'
    })
''')

In [ ]:
run(driver, '''
    CALL gds.pageRank.write('user-feature-projection', {
        relationshipTypes: ['P2P_WITH_SHARED_CARD'],
        maxIterations: 1000,
        writeProperty: 'p2pSharedCardPageRank'
    })
''')

run(driver, '''
    CALL gds.degree.write('user-feature-projection', {
        relationshipTypes: ['P2P_WITH_SHARED_CARD'],
        writeProperty: 'p2pSharedCardDegree'
    })
''')

run(driver, '''
    CALL gds.pageRank.write('user-feature-projection', {
        relationshipTypes: ['P2P_WITH_SHARED_CARD_REVERSE'],
        maxIterations: 1000,
        writeProperty: 'p2pReversedSharedCardPageRank'
    })
''')

run(driver, '''
    CALL gds.degree.write('user-feature-projection', {
        relationshipTypes: ['P2P_WITH_SHARED_CARD_REVERSE'],
        writeProperty: 'p2pReversedSharedCardDegree'
    })
''')

In [ ]:
run(driver, '''
    CALL gds.pageRank.write('user-feature-projection', {
        relationshipTypes: ['P2P'],
        maxIterations: 1000,
        writeProperty: 'p2pSentPageRank'
    })
''')

run(driver, '''
    CALL gds.pageRank.write('user-feature-projection', {
        relationshipTypes: ['P2P'],
        maxIterations: 1000,
        relationshipWeightProperty: 'totalAmount',
        writeProperty: 'p2pSentWeightedPageRank'
    })
''')

run(driver, '''
    CALL gds.degree.write('user-feature-projection', {
        relationshipTypes: ['P2P'],
        relationshipWeightProperty: 'totalAmount',
        writeProperty: 'p2pSentWeightedDegree'
    })
''')

run(driver, '''
    CALL gds.degree.write('user-feature-projection', {
        relationshipTypes: ['P2P'],
        writeProperty: 'p2pSentDegree'
    })
''')

In [ ]:
run(driver, '''
    CALL gds.pageRank.write('user-feature-projection', {
        relationshipTypes: ['P2P_REVERSE'],
        maxIterations: 1000,
        writeProperty: 'p2pReceivedPageRank'
    })
''')

run(driver, '''
    CALL gds.pageRank.write('user-feature-projection', {
        relationshipTypes: ['P2P_REVERSE'],
        maxIterations: 1000,
        relationshipWeightProperty: 'totalAmount',
        writeProperty: 'p2pReceivedWeightedPageRank'
    })
''')

run(driver, '''
    CALL gds.degree.write('user-feature-projection', {
        relationshipTypes: ['P2P_REVERSE'],
        relationshipWeightProperty: 'totalAmount',
        writeProperty: 'p2pReceivedWeightedDegree'
    })
''')

run(driver, '''
    CALL gds.degree.write('user-feature-projection', {
        relationshipTypes: ['P2P_REVERSE'],
        writeProperty: 'p2pReceivedDegree'
    })
''')
clear_graph(driver, 'user-feature-projection')

#### ID Centrality Features

In [ ]:
clear_graph(driver, 'user-id-degree-projection')
run(driver, '''
    CALL gds.graph.create('user-id-degree-projection',
        ['User', 'Card', 'Device', 'IP'],
        ['HAS_CC', 'HAS_IP', 'USED']
    )
''')

run(driver, '''
    CALL gds.degree.mutate('user-id-degree-projection', {
        nodeLabels: ['User', 'Card'],
        relationshipTypes: ['HAS_CC'],
        mutateProperty: 'cardDegree'
    })
''')

run(driver, '''
    CALL gds.degree.mutate('user-id-degree-projection', {
        nodeLabels: ['User', 'Device'],
        relationshipTypes: ['USED'],
        mutateProperty: 'deviceDegree'
    })
''')

run(driver, '''
    CALL gds.degree.mutate('user-id-degree-projection', {
        nodeLabels: ['User', 'IP'],
        relationshipTypes: ['HAS_IP'],
        mutateProperty: 'ipDegree'
    })
''')

run(driver, '''
    CALL gds.graph.writeNodeProperties('user-id-degree-projection',
        ['cardDegree', 'deviceDegree', 'ipDegree'],
        ['User']
    )
''')
clear_graph(driver, 'user-id-degree-projection')

### Machine Learning Training & Evaluation

#### Get and Prepare Data

In [ ]:
%%time
result = run(driver, 'MATCH(u:User) RETURN u')
df = pd.DataFrame([dict(record.get('u')) for record in result])
df

In [ ]:
result = run(driver, 'MATCH(u:User) RETURN u')
df = pd.DataFrame([dict(record.get('u')) for record in result])

X = df[[
    'sharedIdsDegree',
    'p2pSharedCardPageRank',
    'p2pSentPageRank',
    'p2pReceivedWeightedPageRank',
    'p2pReceivedWeightedDegree',
    'ipDegree',
    'cardDegree',
    'deviceDegree',
    'communitySize',
    'partOfCommunity'
]]

y = df.fraudRisk - df.fraudMoneyTransfer

In [ ]:
X

In [ ]:
print(y)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#### Model Training and Evaluation

For purposes of this demo I am going to use a random forest classifier. Other classifiers including logistic regression, SVM, Neural Nets and Boosting variants could work as well. Going into the exact pros and cons of these models is out of scope here. Overall, I like exploring classification with Random Forests since they are relatively robust to feature scaling and collinearity issues and require minimal tuning to get working well.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=500, random_state=0, max_depth=5, bootstrap=True, class_weight='balanced')
clf.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
print('Accuracy of random forrest classifier on test set: {:.2f}'.format(clf.score(X_test, y_test)))
print('\nConfusion Matrix: ')
disp = ConfusionMatrixDisplay.from_predictions(y_test, clf.predict(X_test), display_labels=clf.classes_,
                                               normalize='true', cmap='Greys', colorbar=False)

In [ ]:
from sklearn.metrics import RocCurveDisplay

display = RocCurveDisplay.from_estimator(clf, X_test, y_test, name="RF Model")
_ = display.ax_.set_title("ROC Curve")

In [ ]:
from sklearn.metrics import PrecisionRecallDisplay

y_prob = clf.predict_proba(X_test)
display = PrecisionRecallDisplay.from_predictions(y_test, y_prob[:, 1], name="RF Model")
_ = display.ax_.set_title("Precision-Recall Curve")

Below is a ranked list of the most influential features. Among the most important are the community sizes and the shared ids degree and p2p shared card pageRank.

In [ ]:
from sklearn.inspection import permutation_importance
result = permutation_importance(clf, X_train, y_train, random_state=0)
pd.DataFrame(abs(result['importances_mean']),index=X_train.columns).sort_values(0, ascending=False)

### Investigating Unlabeled High-Probability Fraud Risk Predictions
The labeling from part 2 wasn't perfect. Now that we have trained a machine learning model, investigating user accounts that were predicted as high probability fraud risks despite not being labeled as such by us (ostensible false positives), will bring further insights.

The below commands will isolate some cases from the test set so we can visualize in [Neo4j Bloom](https://neo4j.com/product/bloom/)

In [ ]:
# Retrieve High Probability predictions for non-fraud risk labeled data in the testset
y_test_df = y_test.to_frame(name='cls')
y_test_df['predictedProbability']=y_prob[:, 1]
test_prob_df = y_test_df[(y_test_df.predictedProbability > 0.88) & (y_test_df.cls == 0)].join(df[['guid','wccId', 'communitySize']])
test_prob_df

In [ ]:
#Write back to database for investigation in Bloom
for index, row in test_prob_df.iterrows():
    run(driver, '''
        MATCH(u:User) WHERE u.guid = $guid
        SET u.predictedProbability = $predictedProbability
    ''', params = row.to_dict())

Below are a couple examples from the test set visualized in [Bloom](https://neo4j.com/docs/bloom-user-guide/current/) . The larger, leftmost, green nodes with user icons represent the high probability nodes of interest.

Example 1
<img src="img/predictions-in-bloom-1.png" width="700" >\n

Example 2
<img src="img/predictions-in-bloom-2.png" width="700" >


Perhaps unsurprisingly, they exhibit the P2P with shared card behavior, they also have a relatively large number of credit cards (the median degree centrality on cards is 3). This could potentially be a sign of fraud, though it is hard to know on an anonymized dataset like this. This is where subject matter expert review and iteration comes in. If this behavior turns out to be a clear indicator of fraud, it means we are predicting fraud more proactively before chargebacks take place which is the ideal. In this case, if we re-label these users appropriately and re-train our ML model as more data comes in, we will further improve predictive performance. If, on the other hand, it turns out that some of this behavior is benign, we can adjust the feature engineer and model so the ML learns to rule out such cases which will likewise improve predictive performance and increase our understanding of fraud patterns. Either way, it is a win.

## Clean Up
This section will help clean all the additional graph elements and properties created in the above workflow.

In [ ]:
# delete created relationships
run(driver,'MATCH (:User)-[r:SHARED_IDS]->() DELETE r')
run(driver,'MATCH (:User)-[r:P2P_WITH_SHARED_CARD]->() DELETE r')
run(driver,'MATCH (:User)-[r:SIMILAR_IDS]->() DELETE r')

In [ ]:
# remove created node Labels
run(driver,'MATCH (u:FlaggedUser) REMOVE u:FlaggedUser')
run(driver,'MATCH (u:FraudRiskUser) REMOVE u:FraudRiskUser')
run(driver,'MATCH (u:FraudSharedId) REMOVE u:FraudSharedId')

In [ ]:
# remove created node properties
run(driver,'''
    MATCH (n)
    REMOVE n.wccId,
        n.sharedIdsDegree,
        n.predictedProbability,
        n.partOfCommunity,
        n.p2pSharedCardPageRank,
        n.p2pSharedCardDegree,
        n.p2pSentWeightedPageRank,
        n.p2pSentWeightedDegree,
        n.p2pSentPageRank,
        n.p2pSentDegree,
        n.p2pReversedSharedCardPageRank,
        n.p2pReversedSharedCardDegree,
        n.p2pReceivedWeightedPageRank,
        n.p2pReceivedWeightedDegree,
        n.p2pReceivedPageRank,
        n.p2pReceivedDegree,
        n.louvainCommunityId,
        n.ipDegree,
        n.fraudRiskRatio,
        n.fraudRiskDegree,
        n.fraudRisk,
        n.flaggedRatio,
        n.flaggedDegree,
        n.deviceDegree,
        n.degree,
        n.communitySize,
        n.cardDegree
''')

In [ ]:
# remove created relationship properties
run(driver, 'MATCH ()-[r]->() REMOVE r.inverseDegreeWeight')

In [ ]:
# Remove GDS graph projections
clear_all_graphs(driver)